In [1]:
# First we load our location data and drop any duplicated places within documents
import pandas as pd
import shapely.vectorized
from global_land_mask import globe

import warnings
import time

# Load location data 
places = pd.read_csv('data/clean_places.csv')                                                                                                                                                                                                                                                                                                                                                                                                                              
places = places.drop_duplicates(["doc_id","geonameid"]) 
print(places.shape)
places.head()

/tmp/ipykernel_223004/3951502023.py:10: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  places = pd.read_csv('data/clean_places.csv')


(218096, 13)


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id
0,Shanghai,"[{'start': 162, 'end': 170}]",CHN,0.963741,Shanghai,31.22222,121.45806,CHN,1796236.0,Shanghai,P,PPLA,4963912
5,Aeromonas,"[{'start': 1153, 'end': 1162}]",NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4729030
6,Thailand,"[{'start': 57, 'end': 65}]",THA,0.948191,NaN,15.50000,101.00000,THA,1605651.0,Kingdom of Thailand,A,PCLI,4695222
7,Bangkok,"[{'start': 302, 'end': 309}]",THA,0.961249,Bangkok,13.75398,100.50144,THA,1609350.0,Bangkok,P,PPLC,4695222
9,Israel,"[{'start': 285, 'end': 291}]",ISR,0.948191,NaN,31.50000,34.75000,ISR,294640.0,State of Israel,A,PCLI,4762134


In [2]:
df = pd.read_feather("data/included_studies.feather")
df.head()

,id,title,abstract,publication_year,0 - relevant,2 - 1.02. Changes in temperature,2 - 1.03. Seasonal change,2 - 1.04. Changes in precipitation,2 - 1.06. Climate change (unspecified),2 - 1.07. Other meteorological variables,...,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios,5 - 4.01. Floods and drought,5 - 4.02. Heatwaves,5 - 4.03. Wildfires,5 - 4.04. Other extreme events,5 - 4.05. Extreme cold,6 - 5.01. Pollution,6 - 5.03. Reduced agricultural & aquaculture productivity,6 - 5.04. Reduced labour and physical capacity
0,W2021823358,Projections of heat waves with high impact on ...,"Climate change will result in more intense, mo...",2014.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,W2129447349,Apparent Temperature and Cause-Specific Mortal...,"Temperature, a key climate change indicator, i...",2011.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,W2345489509,Climate Change and the Emergent Epidemic of CK...,Climate change has led to significant rise of ...,2016.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,W2093213412,Children are likely to suffer most from our fo...,BACKGROUND: The periods of fetal and child dev...,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,W615562828,Autochthonous Chikungunya Transmission and Ext...,Background Extreme precipitation events are in...,2015.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import numpy as np
def generate_grid_df(degrees):
    '''
    Generate a dataframe with a grid of of cells degrees x degrees
    '''
    LON = np.linspace(-180+degrees*0.5,180-degrees*0.5,int(360/degrees))
    LAT = np.linspace(-90+degrees*0.5,90-degrees*0.5,int(180/degrees))
    lon_df, lat_df = np.meshgrid(LON,LAT)

    return pd.DataFrame({"LAT": lat_df.ravel(), "LON": lon_df.ravel()})
    
grid_df = generate_grid_df(2.5)
grid_df.head()

,LAT,LON
0,-88.75,-178.75
1,-88.75,-176.25
2,-88.75,-173.75
3,-88.75,-171.25
4,-88.75,-168.75


In [4]:
import cartopy.io.shapereader as shpreader
import geopandas
# First we define a list of the shapefile definitions we want
shpfiles = [
    dict(resolution='50m', category='cultural', name='admin_0_countries'),
    dict(resolution='10m', category='cultural', name='admin_1_states_provinces'),
    dict(resolution='10m', category='physical', name='geography_regions_polys'),
    dict(resolution='10m', category='physical', name='geography_marine_polys')
    #"data/gadm36_1.shp"
]

/home/max/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_223004/1055013544.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.h

In [5]:
# We'll start an empty dataframe to store our shapefile-grid matches
shp_grid_df = pd.DataFrame()

# Now we download the shapefiles and combine into one large geopandas dataframe
shp_df  = None
for shpfilename in shpfiles:
    print(f"reading shapefile {shpfilename}")
    if shp_df is None:
        shp_df = geopandas.read_file(shpreader.natural_earth(**shpfilename))
    else:
        shp_df = shp_df.merge(geopandas.read_file(shpreader.natural_earth(**shpfilename)),how="outer")
    time.sleep(10) # Wait a bit before downloading the next shapefile so we do not make too many requests too quickly

reading shapefile {'resolution': '50m', 'category': 'cultural', 'name': 'admin_0_countries'}
reading shapefile {'resolution': '10m', 'category': 'cultural', 'name': 'admin_1_states_provinces'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_regions_polys'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_marine_polys'}


In [6]:




degrees = 2.5

# We are going to store our shapefile-gridcell index matches here
shp_grid = []

# This is the grid we will work with
yv, xv = np.meshgrid(grid_df.LAT.unique(), grid_df.LON.unique())
for i, place in shp_df.iterrows(): # Now we go through all the shapes
    # show which gridcell centers are contained inside the shape
    # ignore the warning caused by shapely using an old version of numpy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        inplace = shapely.vectorized.contains(place.geometry, xv, yv)
    idx = np.argwhere(inplace)
    # Get the number of cells contained in the shape
    number_cells = idx.size/2
    if number_cells == 0:
        # If we have no cell centers in the shape, get the shape center and the cell which contains it
        c = place.geometry.centroid
        lon = c.x//degrees*degrees+degrees*0.5
        lat = c.y//degrees*degrees+degrees*0.5
        da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
        shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]})
    else:
        for point in idx:
            lon = grid_df.LON.unique()[point[0]]
            lat = grid_df.LAT.unique()[point[1]]
            da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
            shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]}) 

shp_grid_df = pd.DataFrame.from_dict(shp_grid)
    

shp_grid_df.head()

,shpfile_id,grid_df_id
0,0,4114
1,0,3971
2,0,4115
3,0,3972
4,0,4116


In [7]:
feature_mapping = {
    "ADM1": ["Admin-1 scale rank", "Admin-1 aggregation", "Admin-1 minor island"],
    "PCLI": ["Admin-0 country"],
    'MTS': ['Range/mtn'],
    'PLAT': ['Plateau'],
    'PLN': ['Plain'],
    'DSRT': ['Desert'],
    'OCN': ['ocean'],
    'SEA': ['sea', 'bay'],
    'GULF': ['gulf', 'bay'],
    'BAY': ['gulf', 'bay'],
    'CHN': ['channel'],
    'BSNU': ['basin']
}

In [8]:
# We add all the other codes we don't cover with blank shapefile classes
for fcode in places.feature_code.unique():
    if fcode not in feature_mapping:
        feature_mapping[fcode] = []

In [9]:
# To help match, we will rename some shapes so that they are the same as the name in our database
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Altay", case=False)),"name"] = "Altay"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Appalach", case=False)),"name"] = "Appalachian Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("cant", case=False)),"name"] = "Cordillera Cantábrica"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Dabie", case=False)),"name"] = "Dabie Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("EASTERN GHATS", case=False)),"name"] = "Eastern Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("WESTERN GHATS", case=False)),"name"] = "Western Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("kunlun", case=False)),"name"] = "Kalakunlun Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("LEN MOUNTAIN", case=False)),"name"] = "Kölen"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Taihang Mts.", case=False)),"name"] = "Taihang Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Tatra Mts.", case=False)),"name"] = "Tatry"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("TIAN SHAN", case=False)),"name"] = "Tien Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("andes", case=False)),"name"] = "Andes Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("HINDU KUSH", case=False)),"name"] = "Hindū Kush"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Marrah Mts", case=False)),"name"] = "Jabal Marrah"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Lebanon", case=False)),"name"] = "Mount Lebanon"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("KARAKORAM RA", case=False)),"name"] = "Karakorum Shan"

shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Negev", case=False)), "name"] = "Negev"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Atacama", case=False)), "name"] = "Atacama Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("CHIHUAHUAN DESERT", case=False)), "name"] = "Chihuahua Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Lut desert", case=False)), "name"] = "God-e Lut"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("TAKLIMAKAN DESERT", case=False)), "name"] = "Takla Makan Desert"

shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("cumberland", case=False)),["name","featurecla"]
] = ["Cumberland Plateau", "Plain"]
shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("colorado", case=False)),["name","featurecla"]
] = ["San Francisco Plateau", "Plain"]
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("gange", case=False)),"name"] = "Gangetic Plain"
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("north china", case=False)),"name"] = "Huanghuai Pingyuan"

shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("mongol", case=False)), "name"] = "Nei Mongol Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("deccan", case=False)), "name"] = "Deccan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("chota", case=False)), "name"] = "Chota Nāgpur Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("loess", case=False)), "name"] = "Huangtu Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("khorat", case=False)), "name"] = "Khorat Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("tibet", case=False)), "name"] = "Qing Zang Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("polar", case=False)), "name"] = "South Polar Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("YUNGUI", case=False)), "name"] = "Yungui Gaoyuan"

places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains") 
shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains") 

/tmp/ipykernel_223004/2263670162.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains")
/tmp/ipykernel_223004/2263670162.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
  shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains")


In [10]:
# We'll start off with an empty dataframe
shp_df_matches = pd.DataFrame()

# And define the grid box size
degrees = 2.5

# Now we want to look through the feature type dictionary we had before
for place_key, shpfile_keys in feature_mapping.items():
    
    print(place_key)
    
    # We can get all the places and all the shapes
    feature_places = places.loc[places["feature_code"]==place_key]
    feature_shapes = shp_df.loc[shp_df["featurecla"].isin(shpfile_keys)]
    
    # And loop through the places
    for place, group in feature_places.groupby("place_name"):
        # If we don't have a shapefile feature type we just take the grid cell containing the point
        if not shpfile_keys:
            shp_id = None # we set shp_id to None, round the coordinates, and take the gridcell which has these coordinates
            lon = group.lon.values[0]//degrees*degrees+degrees*0.5
            lat = group.lat.values[0]//degrees*degrees+degrees*0.5
            grid_df_ids = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)].index

        else:
            # Otherwise, we are going to try to find the places which match
            place_shapes = pd.DataFrame()
            # First we will try by geoname ids, if we have these
            if feature_shapes["gn_id"].sum() > 0:
                place_shapes = feature_shapes.loc[
                    (feature_shapes["gn_id"]==group.geonameid.values[0])
                ]
            # Then we try by country code, if we are dealing with a country
            if place_shapes.shape[0]==0 and place_key=="PCLI":
                place_shapes = feature_shapes.loc[
                    feature_shapes["ADM0_A3"] == group.country_predicted.values[0]
                ]
            # Then we try by name
            if place_shapes.shape[0]==0:       
                place_shapes = feature_shapes.loc[
                    (feature_shapes.name == place)
                ]    

            if place_shapes.shape[0]==0:
                continue # These are the places we cannot match, we need to develop other strategies for these, e.g. using other shapefile libraries
    #            if key=="ADM1": # can also try using gadm data, which we need to download separately
    #                 place_shapes = adm1shps_alt[
    #                     (adm1shps_alt["NAME_1"]==group.place_name.values[0]) |
    #                     (adm1shps_alt["VARNAME_1"].str.contains(group.place_name.values[0]))
    #                 ]
            else:
                shp_id = place_shapes.index[0]
                grid_df_ids = shp_grid_df.loc[
                    (shp_grid_df["shpfile_id"]==shp_id),
                    "grid_df_id"
                ]
        # For each document with this place, we add a row for each grid cell index matching the place
        for did in group.doc_id.unique():
            shp_df_matches = pd.concat([
                shp_df_matches,
                pd.DataFrame.from_dict({
                    "grid_df_id": grid_df_ids,
                    "doc_id": [did] * len(grid_df_ids),
                    "shp_id": [shp_id] * len(grid_df_ids),
                    "place": place
                })
            ])
            
            
print(shp_df_matches.shape)
shp_df_matches.head()

ADM1
PCLI
MTS
PLAT
PLN
DSRT
OCN
SEA
GULF
BAY
CHN
BSNU
PPLA
nan
PPLC
ADM3
CONT
AREA
ADM2
ISL
PPLA3
AIRP
RGN
PPL
PCLD
BLDG
PPLA2
HTL
PPLX
LIBR
DPR
ADMD
STM
ISLS
CTHSE
PRK
HSP
CMTY
DIP
PPLL
CH
STNB
RSV
SHRN
MN
ADM4
ITTR
SCHC
PEN
MUS
LK
SCH
COVE
PCLS
TOWR
MSSN
ADM1H
LNDF
PPLA4
RESF
AIRH
RSD
MT
RSVT
PK
CNL
PMPW
ST
RGNE
PPLQ
STNM
CST
VLC
HSTS
FCL
USGE
PCLF
FRST
DAM
PPLF
RESN
PCLH
BANK
AIRF
HMSD
LCTY
DLTA
SWT
OILF
PT
WTRH
EST
AIRQ
VAL
BCH
PCLIX
UPLD
HSE
ADM3H
RD
MNMT
CMPRF
RSTN
RECG
HLL
GAP
STMC
PPLG
DEVH
MALL
CMP
AGRF
FRM
GLCR
SALT
PPLW
WLL
ADM2H
TMB
PPLH
HSPC
RES
ADMF
WAD
ZN
PPLA5
MFG
PO
UNIV
HLLS
MSQE
INDS
TRL
FLDI
BUSTP
ADM4H
RR
TRIG
INSM
RDJCT
GHSE
STDM
RGNH
BLDO
WRCK
LTHSE
STMI
PS
DTCHI
RET
SCRP
PYR
BTL
PSH
BUSTN
COMC
CAPE
SPNG
LOCK
PND
STMD
DCK
FT
SQR
STNE
ATOL
CTRS
STMM
HUT
HBR
LEV
DUNE
RDGE
UNIP
TMPL
SHSE
CNLA
CLDA
RKS
CMPO
RDGU
BDG
LKN
TRGU
ADM5
SMSU
MSTY
LAVA
RESV
RK
STMB
RECR
MAR
MTRO
TRB
SLP
RNCH
ANS
RSTNQ
STRT
FLLS
PASS
LDGU
SCHT
PAL
MRSHN
CHNM
SMU
AIRB
ML
RSRT
RSTP
AIRS
PRT
EST

,grid_df_id,doc_id,shp_id,place
5608,6281,5249722,1710,'asir region
5608,6281,5110789,1710,'asir region
5608,6281,3927578,1710,'asir region
5608,6281,3313720,1710,'asir region
5608,6281,W2291458255,1710,'asir region


In [11]:
shp_df_matches.to_csv("data/shp_df_matches.csv",index=False)

In [12]:
import rioxarray

rds = rioxarray.open_rasterio("data/gpw_v4_population_count_rev11_2020_1_deg.asc",
)

rds = rds.squeeze().drop("spatial_ref").drop("band")
rds.name = "population"
pop_df = rds.to_dataframe().reset_index()

pop_df['LAT'] = pop_df['y']//2.5*2.5+1.25
pop_df['LON'] = pop_df['x']//2.5*2.5+1.25
pop_df.loc[pop_df['population']==-9999,"population"]=np.NaN
pop_df = pop_df.groupby(["LAT","LON"])['population'].sum().reset_index()
pop_df

,LAT,LON,population
0,-88.75,-178.75,0.0
1,-88.75,-176.25,0.0
2,-88.75,-173.75,0.0
3,-88.75,-171.25,0.0
4,-88.75,-168.75,0.0
...,...,...,...
10363,88.75,168.75,0.0
10364,88.75,171.25,0.0
10365,88.75,173.75,0.0
10366,88.75,176.25,0.0


In [13]:
import math
def area_cell(lat, lon, degrees): 
    # calculate the area of a gridcell given the center lat and lon and the size in degrees
    if lon <0:
        lon+=360
    R = 6371
    f0 = math.radians(lat-degrees*0.5)
    f1 = math.radians(lat+degrees*0.5)
    l0 = math.radians(lon-degrees*0.5)
    l1 = math.radians(lon+degrees*0.5)

    return (math.sin(f1)-math.sin(f0)) * (l1 - l0) * R**2


grid_df['area'] = grid_df.apply(lambda x: area_cell(x['LAT'], x['LON'], 2.5), axis=1)

In [14]:
np.linspace(0,2.5,5)

array([0.   , 0.625, 1.25 , 1.875, 2.5  ])

In [15]:
land_masks = []
n = 5
land_array = np.empty((grid_df.shape[0], n*n))
i = 0
for x in np.linspace(0,2.5,n):
    for y in np.linspace(0,2.5,n):
        land_array[:,i] = globe.is_land(grid_df.LAT+y-1.25, grid_df.LON+x-1.25)
        i+=1
        
land_mask = land_array.sum(axis=1)>0

In [16]:
grid_df['is_land'] = land_mask
grid_df.loc[grid_df['LAT']<-60,'is_land'] = False

In [17]:
grid_df.is_land.sum()

3311

In [18]:
ncc_data = pd.read_csv("data/2_merged_da_data.csv")
grid_df = grid_df.merge(
    ncc_data[["LAT","LON","updated_precip","updated_temp"]].rename(columns={"updated_precip": "precip_da", "updated_temp": "temp_da"})
).merge(pop_df)
grid_df.reset_index().to_csv("data/grid_df.csv", index=False)
grid_df.head()

,LAT,LON,area,is_land,precip_da,temp_da,population
0,-88.75,-178.75,1685.654015,False,NaN,NaN,0.0
1,-88.75,-176.25,1685.654015,False,NaN,NaN,0.0
2,-88.75,-173.75,1685.654015,False,NaN,NaN,0.0
3,-88.75,-171.25,1685.654015,False,NaN,NaN,0.0
4,-88.75,-168.75,1685.654015,False,NaN,NaN,0.0


In [19]:
grid_df = pd.read_csv("data/grid_df.csv")

In [20]:
import rioxarray

rds = rioxarray.open_rasterio("data/gpw_v4_population_count_rev11_2020_1_deg.asc",
)

rds = rds.squeeze().drop("spatial_ref").drop("band")
rds.name = "population"
pop_df = rds.to_dataframe().reset_index()

pop_df['LAT'] = pop_df['y']//2.5*2.5+1.25
pop_df['LON'] = pop_df['x']//2.5*2.5+1.25
pop_df.loc[pop_df['population']==-9999,"population"]=np.NaN
pop_df = pop_df.merge(grid_df, how="left")

In [21]:
pop_df

,y,x,population,LAT,LON,index,area,is_land,precip_da,temp_da
0,89.5,-179.5,NaN,88.75,-178.75,NaN,NaN,NaN,NaN,NaN
1,89.5,-178.5,NaN,88.75,-178.75,NaN,NaN,NaN,NaN,NaN
2,89.5,-177.5,NaN,88.75,-176.25,NaN,NaN,NaN,NaN,NaN
3,89.5,-176.5,NaN,88.75,-176.25,NaN,NaN,NaN,NaN,NaN
4,89.5,-175.5,NaN,88.75,-176.25,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
64795,-89.5,175.5,NaN,-88.75,176.25,NaN,NaN,NaN,NaN,NaN
64796,-89.5,176.5,NaN,-88.75,176.25,NaN,NaN,NaN,NaN,NaN
64797,-89.5,177.5,NaN,-88.75,178.75,NaN,NaN,NaN,NaN,NaN
64798,-89.5,178.5,NaN,-88.75,178.75,NaN,NaN,NaN,NaN,NaN


In [22]:
countries = geopandas.read_file(shpreader.natural_earth(
    **dict(resolution='50m', category='cultural', name='admin_0_countries')
))
countries.columns

Index(['featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT', 'SOV_A3',
       'ADM0_DIF', 'LEVEL', 'TYPE', 'ADMIN', 'ADM0_A3', 'GEOU_DIF', 'GEOUNIT',
       'GU_A3', 'SU_DIF', 'SUBUNIT', 'SU_A3', 'BRK_DIFF', 'NAME', 'NAME_LONG',
       'BRK_A3', 'BRK_NAME', 'BRK_GROUP', 'ABBREV', 'POSTAL', 'FORMAL_EN',
       'FORMAL_FR', 'NAME_CIAWF', 'NOTE_ADM0', 'NOTE_BRK', 'NAME_SORT',
       'NAME_ALT', 'MAPCOLOR7', 'MAPCOLOR8', 'MAPCOLOR9', 'MAPCOLOR13',
       'POP_EST', 'POP_RANK', 'GDP_MD_EST', 'POP_YEAR', 'LASTCENSUS',
       'GDP_YEAR', 'ECONOMY', 'INCOME_GRP', 'WIKIPEDIA', 'FIPS_10_', 'ISO_A2',
       'ISO_A3', 'ISO_A3_EH', 'ISO_N3', 'UN_A3', 'WB_A2', 'WB_A3', 'WOE_ID',
       'WOE_ID_EH', 'WOE_NOTE', 'ADM0_A3_IS', 'ADM0_A3_US', 'ADM0_A3_UN',
       'ADM0_A3_WB', 'CONTINENT', 'REGION_UN', 'SUBREGION', 'REGION_WB',
       'NAME_LEN', 'LONG_LEN', 'ABBREV_LEN', 'TINY', 'HOMEPART', 'MIN_ZOOM',
       'MIN_LABEL', 'MAX_LABEL', 'NE_ID', 'WIKIDATAID', 'NAME_AR', 'NAME_BN',
       'NAME_DE', 'NAME_

In [23]:
countries.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,NAME_KO,NAME_NL,NAME_PL,NAME_PT,NAME_RU,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,geometry
0,Admin-0 country,1,3,Zimbabwe,ZWE,0,2,Sovereign country,Zimbabwe,ZWE,...,ì§ë°ë¸ì¨,Zimbabwe,Zimbabwe,ZimbÃ¡bue,ÐÐ¸Ð¼Ð±Ð°Ð±Ð²Ðµ,Zimbabwe,Zimbabve,Zimbabwe,è¾å·´å¨,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Admin-0 country,1,3,Zambia,ZMB,0,2,Sovereign country,Zambia,ZMB,...,ì ë¹ì,Zambia,Zambia,ZÃ¢mbia,ÐÐ°Ð¼Ð±Ð¸Ñ,Zambia,Zambiya,Zambia,èµæ¯äº,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."
2,Admin-0 country,1,3,Yemen,YEM,0,2,Sovereign country,Yemen,YEM,...,ìë©,Jemen,Jemen,IÃ©men,ÐÐµÐ¼ÐµÐ½,Jemen,Yemen,Yemen,ä¹é¨,"MULTIPOLYGON (((53.08564 16.64839, 52.58145 16..."
3,Admin-0 country,3,2,Vietnam,VNM,0,2,Sovereign country,Vietnam,VNM,...,ë² í¸ë¨,Vietnam,Wietnam,Vietname,ÐÑÐµÑÐ½Ð°Ð¼,Vietnam,Vietnam,Viá»t Nam,è¶å,"MULTIPOLYGON (((104.06396 10.39082, 104.08301 ..."
4,Admin-0 country,5,3,Venezuela,VEN,0,2,Sovereign country,Venezuela,VEN,...,ë² ë¤ììë¼,Venezuela,Wenezuela,Venezuela,ÐÐµÐ½ÐµÑÑÑÐ»Ð°,Venezuela,Venezuela,Venezuela,å§å §çæ,"MULTIPOLYGON (((-60.82119 9.13838, -60.94141 9..."


In [24]:
degrees = 1

# We are going to store our shapefile-gridcell index matches here
shp_grid = []

# This is the grid we will work with
yv, xv = np.meshgrid(pop_df.y.unique(), pop_df.x.unique())
for i, place in countries.iterrows(): # Now we go through all the shapes
    # show which gridcell centers are contained inside the shape
    # ignore the warning caused by shapely using an old version of numpy
    for offset in [-degrees*0.5, degrees*0.5, 0]:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            inplace = shapely.vectorized.contains(place.geometry, xv+offset, yv+offset)
        idx = np.argwhere(inplace)
        # Get the number of cells contained in the shape
        number_cells = idx.size/2
        if number_cells == 0:
            # If we have no cell centers in the shape, get the shape center and the cell which contains it
            c = place.geometry.centroid
            lon = c.x//degrees*degrees+degrees*0.5
            lat = c.y//degrees*degrees+degrees*0.5
            pop_df.loc[(pop_df['x']==lon) & (pop_df['y']==lat),"country"] = place["ISO_A3"]
        else:
            for point in idx:
                lon = pop_df.x.unique()[point[0]]
                lat = pop_df.y.unique()[point[1]]
                pop_df.loc[(pop_df['x']==lon) & (pop_df['y']==lat),"country"] = place["ISO_A3"]


In [25]:
pop_df[pd.isna(pop_df["country"])]["population"].sum()

174186820.0

In [ ]:
from shapely.geometry import Polygon
polygons = []
for x in pop_df.x:
    for y in pop_df.y:
        polygons.append(Polygon([
            (x-degrees*0.5,y-degrees*0.5), (x+degrees*0.5, y-degrees*0.5), 
            (x+degrees*0.5,y+degrees*0.5), (x-degrees*0.5, y+degrees*0.5)
        ]))

grid = gpd.GeoDataFrame({'geometry':polygons})
grid

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
from matplotlib import colormaps

fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection=ccrs.Robinson())
ax.coastlines(lw=0.2)

shape = (len(pop_df.y.unique()), len(pop_df.x.unique()))

pop_df["n"] = pop_df.population
pop_df.loc[pd.notna(pop_df["country"]),"n"] = 0

n = np.array(pop_df.n).reshape(shape)

mesh=ax.pcolormesh(
    pop_df.x.unique(), 
    pop_df.y.unique(), 
    n, 
    cmap=colormaps['magma_r'],
    norm=mpl.colors.LogNorm(),
    transform=ccrs.PlateCarree(),
)
